# Loan Approval Prediction

## Keras - 2 Attempts

## Attempt 1

In [2]:
# 1. Reproducibility
import random
random.seed(42)
import numpy as np
np.random.seed(42)
import tensorflow as tf
tf.random.set_seed(42)

# 2. Imports
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import json, time

# 3. Data Loading & Target Encoding
# Load datasets
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
submission_template = pd.read_csv('sample_submission.csv')

# Infer id_col and target_columns from sample submission
cols = submission_template.columns.tolist()
id_col = cols[0]
target_col = cols[1]

# Label encode target (binary classification)
le = LabelEncoder().fit(train_df[target_col].astype(str))
y_train = le.transform(train_df[target_col].astype(str)).astype(int)

# Prepare feature matrices
drop_cols = [id_col, target_col]
X = train_df.drop(columns=drop_cols, errors='ignore')
X_test = test_df.drop(columns=drop_cols, errors='ignore')
test_ids = test_df[id_col]

# 4. Feature Engineering: drop all-missing, high-cardinality categorical
# Drop columns with all missing
X = X.dropna(axis=1, how='all')
X_test = X_test[X.columns]
# Identify categorical and drop if cardinality >50
cat_cols = X.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()
high_card = [c for c in cat_cols if X[c].nunique() > 50]
X = X.drop(columns=high_card)
X_test = X_test.drop(columns=high_card)

# 5. Preprocessing Pipeline
d_numeric = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_features = X.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, d_numeric),
    ('cat', categorical_transformer, cat_features)
])

# Fit & transform
X_train_proc = preprocessor.fit_transform(X)
X_test_proc = preprocessor.transform(X_test)

# 6. Model Architecture for small dataset (n_features <100)
n_features = X_train_proc.shape[1]
inputs = tf.keras.Input(shape=(n_features,))
x = tf.keras.layers.Dense(min(n_features*2, 128), activation='relu')(inputs)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(min(n_features, 64), activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

# 7. Compile model
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

# 8. Callbacks & Training
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)
]
start_time = time.time()
history = model.fit(
    X_train_proc, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=128,
    callbacks=callbacks,
    verbose=2
)
duration = time.time() - start_time

# 9. Evaluation & Logging
results = {
    'training_accuracy': history.history['accuracy'][-1],
    'training_loss': history.history['loss'][-1],
    'validation_accuracy': history.history['val_accuracy'][-1],
    'validation_loss': history.history['val_loss'][-1]
}
with open('results.json', 'w') as f:
    json.dump(results, f)

# 10. Prediction & Submission
raw_preds = model.predict(X_test_proc)
probs = raw_preds.flatten()
final = (probs > 0.5).astype(int)
submission = pd.DataFrame({id_col: test_ids.reset_index(drop=True), target_col: final})
submission.to_csv('submission_result.csv', index=False)

I0000 00:00:1752150252.513595 1676548 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38353 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:04:00.0, compute capability: 8.0


Epoch 1/100


I0000 00:00:1752150254.949959 1676716 service.cc:152] XLA service 0x7b914c00ab40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1752150254.949984 1676716 service.cc:160]   StreamExecutor device (0): NVIDIA A100-SXM4-40GB, Compute Capability 8.0
2025-07-10 12:24:15.002702: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1752150255.188838 1676716 cuda_dnn.cc:529] Loaded cuDNN version 90501
I0000 00:00:1752150256.512027 1676716 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


367/367 - 9s - 23ms/step - accuracy: 0.8905 - auc: 0.8591 - loss: 0.2903 - val_accuracy: 0.9246 - val_auc: 0.9185 - val_loss: 0.2104
Epoch 2/100


367/367 - 2s - 4ms/step - accuracy: 0.9221 - auc: 0.9082 - loss: 0.2238 - val_accuracy: 0.9345 - val_auc: 0.9265 - val_loss: 0.1943
Epoch 3/100


...

367/367 - 2s - 4ms/step - accuracy: 0.9479 - auc: 0.9297 - loss: 0.1763 - val_accuracy: 0.9515 - val_auc: 0.9352 - val_loss: 0.1671
Epoch 53/100
367/367 - 1s - 4ms/step - accuracy: 0.9485 - auc: 0.9309 - loss: 0.1745 - val_accuracy: 0.9512 - val_auc: 0.9356 - val_loss: 0.1671
Epoch 54/100
367/367 - 2s - 4ms/step - accuracy: 0.9477 - auc: 0.9301 - loss: 0.1756 - val_accuracy: 0.9516 - val_auc: 0.9349 - val_loss: 0.1675
Epoch 55/100
367/367 - 2s - 4ms/step - accuracy: 0.9486 - auc: 0.9305 - loss: 0.1753 - val_accuracy: 0.9513 - val_auc: 0.9353 - val_loss: 0.1672
Epoch 56/100
367/367 - 1s - 4ms/step - accuracy: 0.9484 - auc: 0.9302 - loss: 0.1755 - val_accuracy: 0.9507 - val_auc: 0.9348 - val_loss: 0.1675
Epoch 57/100
367/367 - 1s - 4ms/step - accuracy: 0.9480 - auc: 0.9314 - loss: 0.1751 - val_accuracy: 0.9513 - val_auc: 0.9344 - val_loss: 0.1679
Epoch 58/100
367/367 - 1s - 4ms/step - accuracy: 0.9483 - auc: 0.9312 - loss: 0.1748 - val_accuracy: 0.9515 - val_auc: 0.9350 - val_loss: 0.167

367/367 - 1s - 4ms/step - accuracy: 0.9485 - auc: 0.9303 - loss: 0.1756 - val_accuracy: 0.9512 - val_auc: 0.9347 - val_loss: 0.1670
Epoch 62/100
367/367 - 1s - 4ms/step - accuracy: 0.9494 - auc: 0.9314 - loss: 0.1731 - val_accuracy: 0.9508 - val_auc: 0.9353 - val_loss: 0.1671
Epoch 63/100


367/367 - 2s - 4ms/step - accuracy: 0.9487 - auc: 0.9319 - loss: 0.1733 - val_accuracy: 0.9515 - val_auc: 0.9351 - val_loss: 0.1665
Epoch 64/100
367/367 - 1s - 4ms/step - accuracy: 0.9492 - auc: 0.9322 - loss: 0.1736 - val_accuracy: 0.9512 - val_auc: 0.9348 - val_loss: 0.1668
Epoch 65/100
367/367 - 1s - 4ms/step - accuracy: 0.9486 - auc: 0.9319 - loss: 0.1732 - val_accuracy: 0.9500 - val_auc: 0.9350 - val_loss: 0.1676
Epoch 66/100
367/367 - 1s - 4ms/step - accuracy: 0.9491 - auc: 0.9313 - loss: 0.1738 - val_accuracy: 0.9510 - val_auc: 0.9343 - val_loss: 0.1676
Epoch 67/100
367/367 - 2s - 4ms/step - accuracy: 0.9492 - auc: 0.9319 - loss: 0.1731 - val_accuracy: 0.9516 - val_auc: 0.9353 - val_loss: 0.1670
Epoch 68/100
367/367 - 1s - 4ms/step - accuracy: 0.9488 - auc: 0.9311 - loss: 0.1742 - val_accuracy: 0.9518 - val_auc: 0.9343 - val_loss: 0.1672
Epoch 69/100
367/367 - 2s - 4ms/step - accuracy: 0.9488 - auc: 0.9315 - loss: 0.1733 - val_accuracy: 0.9514 - val_auc: 0.9347 - val_loss: 0.166

367/367 - 2s - 4ms/step - accuracy: 0.9496 - auc: 0.9318 - loss: 0.1725 - val_accuracy: 0.9518 - val_auc: 0.9351 - val_loss: 0.1664
Epoch 72/100
367/367 - 1s - 4ms/step - accuracy: 0.9490 - auc: 0.9315 - loss: 0.1733 - val_accuracy: 0.9509 - val_auc: 0.9355 - val_loss: 0.1671
Epoch 73/100
367/367 - 2s - 4ms/step - accuracy: 0.9488 - auc: 0.9320 - loss: 0.1734 - val_accuracy: 0.9511 - val_auc: 0.9347 - val_loss: 0.1675
Epoch 74/100
367/367 - 1s - 4ms/step - accuracy: 0.9493 - auc: 0.9335 - loss: 0.1710 - val_accuracy: 0.9512 - val_auc: 0.9353 - val_loss: 0.1673
Epoch 75/100
367/367 - 1s - 4ms/step - accuracy: 0.9492 - auc: 0.9313 - loss: 0.1728 - val_accuracy: 0.9512 - val_auc: 0.9354 - val_loss: 0.1665
Epoch 76/100
367/367 - 2s - 4ms/step - accuracy: 0.9490 - auc: 0.9317 - loss: 0.1734 - val_accuracy: 0.9511 - val_auc: 0.9353 - val_loss: 0.1669
Epoch 77/100
367/367 - 1s - 4ms/step - accuracy: 0.9494 - auc: 0.9320 - loss: 0.1727 - val_accuracy: 0.9513 - val_auc: 0.9349 - val_loss: 0.166

367/367 - 2s - 4ms/step - accuracy: 0.9492 - auc: 0.9326 - loss: 0.1726 - val_accuracy: 0.9516 - val_auc: 0.9344 - val_loss: 0.1663
Epoch 82/100
367/367 - 1s - 4ms/step - accuracy: 0.9502 - auc: 0.9318 - loss: 0.1724 - val_accuracy: 0.9516 - val_auc: 0.9344 - val_loss: 0.1668
Epoch 83/100
367/367 - 1s - 4ms/step - accuracy: 0.9493 - auc: 0.9321 - loss: 0.1726 - val_accuracy: 0.9507 - val_auc: 0.9336 - val_loss: 0.1675
Epoch 84/100
367/367 - 1s - 4ms/step - accuracy: 0.9496 - auc: 0.9333 - loss: 0.1710 - val_accuracy: 0.9509 - val_auc: 0.9343 - val_loss: 0.1673
Epoch 85/100
367/367 - 1s - 4ms/step - accuracy: 0.9494 - auc: 0.9330 - loss: 0.1716 - val_accuracy: 0.9512 - val_auc: 0.9347 - val_loss: 0.1667
Epoch 86/100
367/367 - 1s - 3ms/step - accuracy: 0.9489 - auc: 0.9334 - loss: 0.1716 - val_accuracy: 0.9518 - val_auc: 0.9343 - val_loss: 0.1669
Epoch 87/100
367/367 - 1s - 3ms/step - accuracy: 0.9492 - auc: 0.9338 - loss: 0.1715 - val_accuracy: 0.9514 - val_auc: 0.9344 - val_loss: 0.166

In [3]:
print(duration)

137.25703406333923


## Keras Tuner - 3 Attempt

## Attempt 1

In [1]:
# 1. Reproducibility
import random
random.seed(42)
import numpy as np
np.random.seed(42)
import tensorflow as tf
tf.random.set_seed(42)

# 2. Imports
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import json, time
import keras_tuner as kt
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model

# 3. Data Loading & Target Encoding
# Load datasets
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
submission_template = pd.read_csv('sample_submission.csv')

# Infer id_col and target_columns from sample submission
cols = submission_template.columns.tolist()
id_col = cols[0]
target_col = cols[1]

# Label encode target (binary classification)
le = LabelEncoder().fit(train_df[target_col].astype(str))
y_train = le.transform(train_df[target_col].astype(str)).astype(int)

# Prepare feature matrices
drop_cols = [id_col, target_col]
X = train_df.drop(columns=drop_cols, errors='ignore')
X_test = test_df.drop(columns=drop_cols, errors='ignore')
test_ids = test_df[id_col]

# 4. Feature Engineering: drop all-missing, high-cardinality categorical
# Drop columns with all missing
X = X.dropna(axis=1, how='all')
X_test = X_test[X.columns]
# Identify categorical and drop if cardinality >50
cat_cols = X.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()
high_card = [c for c in cat_cols if X[c].nunique() > 50]
X = X.drop(columns=high_card)
X_test = X_test.drop(columns=high_card)

# 5. Preprocessing Pipeline
d_numeric = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_features = X.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, d_numeric),
    ('cat', categorical_transformer, cat_features)
])

# Fit & transform
X_train_proc = preprocessor.fit_transform(X)
X_test_proc = preprocessor.transform(X_test)

# Split the data for validation
X_train, X_val, y_train, y_val = train_test_split(X_train_proc, y_train, test_size=0.2, random_state=42)

# 6. Model Architecture for small dataset (n_features <100)
n_features = X_train_proc.shape[1]

# Keras-Tuner Model Definition
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

class MyHyperModel(kt.HyperModel):
    def build(self, hp):
        layers = hp.Int('layers', 2, 8)
        units = hp.Int('units', 64, 1024, step=64)
        act = hp.Choice('activation', ['relu'])
        drop = hp.Float('dropout', 0.0, 0.5, step=0.1)
        opt = hp.Choice('optimizer', ['adam'])
        lr = hp.Float('learning_rate', 1e-5, 0.01, sampling='log')

        inputs = Input(shape=(n_features,))
        x = inputs
        for _ in range(layers):
            x = Dense(units, activation=act)(x)
            x = Dropout(drop)(x)
        outputs = Dense(1, activation='sigmoid')(x)
        model = Model(inputs, outputs)
        model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
        return model

# Tuner
bs = 32  # Example batch size
ep = 20  # Example epochs

tuner = kt.BayesianOptimization(
    MyHyperModel(),
    objective='val_loss',
    max_trials=10,
    executions_per_trial=1,
    seed=42,
    overwrite=True,
    project_name='bayesian_tuner'
)

tuner.search(
    X_train_proc, y_train,
    validation_data=(X_val, y_val),
    batch_size=bs, epochs=ep,
    callbacks=[early_stopping, checkpoint]
)

model = tuner.hypermodel.build(tuner.get_best_hyperparameters(1)[0])

start_time = time.time()
history = model.fit(
    X_train, y_train,  # Fixed the input data to match the training labels
    validation_data=(X_val, y_val),
    epochs=100, batch_size=bs,
    callbacks=[early_stopping, checkpoint],
    verbose=2
)
end_time = time.time()

# 9. Evaluation & Logging
results = {
    'training_accuracy': history.history['accuracy'][-1],
    'training_loss': history.history['loss'][-1],
    'validation_accuracy': history.history['val_accuracy'][-1],
    'validation_loss': history.history['val_loss'][-1]
}
with open('results.json', 'w') as f:
    json.dump(results, f)

# 10. Prediction & Submission
raw_preds = model.predict(X_test_proc)
probs = raw_preds.flatten()
final = (probs > 0.5).astype(int)
submission = pd.DataFrame({id_col: test_ids.reset_index(drop=True), target_col: final})
submission.to_csv('submission_result.csv', index=False)

Trial 1 Complete [00h 00m 00s]

Best val_loss So Far: None
Total elapsed time: 00h 00m 01s

Search: Running Trial #2

Value             |Best Value So Far |Hyperparameter
8                 |6                 |layers
128               |64                |units
relu              |relu              |activation
0                 |0.1               |dropout
adam              |adam              |optimizer
4.705e-05         |0.00011364        |learning_rate



Traceback (most recent call last):
  File "/home/exouser/.local/lib/python3.12/site-packages/keras_tuner/src/engine/base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/home/exouser/.local/lib/python3.12/site-packages/keras_tuner/src/engine/base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/exouser/.local/lib/python3.12/site-packages/keras_tuner/src/engine/tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/exouser/.local/lib/python3.12/site-packages/keras_tuner/src/engine/tuner.py", line 233, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
 

RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "/home/exouser/.local/lib/python3.12/site-packages/keras_tuner/src/engine/base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/home/exouser/.local/lib/python3.12/site-packages/keras_tuner/src/engine/base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/exouser/.local/lib/python3.12/site-packages/keras_tuner/src/engine/tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/exouser/.local/lib/python3.12/site-packages/keras_tuner/src/engine/tuner.py", line 233, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, **k

## Attempt 2 - Nothing changed


In [ ]:
# 1. Reproducibility
import random
random.seed(42)
import numpy as np
np.random.seed(42)
import tensorflow as tf
tf.random.set_seed(42)

# 2. Imports
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import json, time
import keras_tuner as kt
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model

# 3. Data Loading & Target Encoding
# Load datasets
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
submission_template = pd.read_csv('sample_submission.csv')

# Infer id_col and target_columns from sample submission
cols = submission_template.columns.tolist()
id_col = cols[0]
target_col = cols[1]

# Label encode target (binary classification)
le = LabelEncoder().fit(train_df[target_col].astype(str))
y_train = le.transform(train_df[target_col].astype(str)).astype(int)

# Prepare feature matrices
drop_cols = [id_col, target_col]
X = train_df.drop(columns=drop_cols, errors='ignore')
X_test = test_df.drop(columns=drop_cols, errors='ignore')
test_ids = test_df[id_col]

# 4. Feature Engineering: drop all-missing, high-cardinality categorical
# Drop columns with all missing
X = X.dropna(axis=1, how='all')
X_test = X_test[X.columns]
# Identify categorical and drop if cardinality >50
cat_cols = X.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()
high_card = [c for c in cat_cols if X[c].nunique() > 50]
X = X.drop(columns=high_card)
X_test = X_test.drop(columns=high_card)

# 5. Preprocessing Pipeline
d_numeric = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_features = X.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, d_numeric),
    ('cat', categorical_transformer, cat_features)
])

# Fit & transform
X_train_proc = preprocessor.fit_transform(X)
X_test_proc = preprocessor.transform(X_test)

# Split the data for validation
X_train, X_val, y_train, y_val = train_test_split(X_train_proc, y_train, test_size=0.2, random_state=42)

# 6. Model Architecture for small dataset (n_features <100)
n_features = X_train_proc.shape[1]

# Keras-Tuner Model Definition
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

class MyHyperModel(kt.HyperModel):
    def build(self, hp):
        layers = hp.Int('layers', 2, 8)
        units = hp.Int('units', 64, 1024, step=64)
        act = hp.Choice('activation', ['relu'])
        drop = hp.Float('dropout', 0.0, 0.5, step=0.1)
        opt = hp.Choice('optimizer', ['adam'])
        lr = hp.Float('learning_rate', 1e-5, 0.01, sampling='log')

        inputs = Input(shape=(n_features,))
        x = inputs
        for _ in range(layers):
            x = Dense(units, activation=act)(x)
            x = Dropout(drop)(x)
        outputs = Dense(1, activation='sigmoid')(x)
        model = Model(inputs, outputs)
        model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
        return model

# Tuner
bs = 32  # Example batch size
ep = 20  # Example epochs

tuner = kt.BayesianOptimization(
    MyHyperModel(),
    objective='val_loss',
    max_trials=10,
    executions_per_trial=1,
    seed=42,
    overwrite=True,
    project_name='bayesian_tuner'
)

tuner.search(
    X_train_proc, y_train,
    validation_data=(X_val, y_val),
    batch_size=bs, epochs=ep,
    callbacks=[early_stopping, checkpoint]
)

model = tuner.hypermodel.build(tuner.get_best_hyperparameters(1)[0])

start_time = time.time()
history = model.fit(
    X_train, y_train,  # Fixed the input data to match the training labels
    validation_data=(X_val, y_val),
    epochs=100, batch_size=bs,
    callbacks=[early_stopping, checkpoint],
    verbose=2
)
end_time = time.time()

# 9. Evaluation & Logging
results = {
    'training_accuracy': history.history['accuracy'][-1],
    'training_loss': history.history['loss'][-1],
    'validation_accuracy': history.history['val_accuracy'][-1],
    'validation_loss': history.history['val_loss'][-1]
}
with open('results.json', 'w') as f:
    json.dump(results, f)

# 10. Prediction & Submission
raw_preds = model.predict(X_test_proc)
probs = raw_preds.flatten()
final = (probs > 0.5).astype(int)
submission = pd.DataFrame({id_col: test_ids.reset_index(drop=True), target_col: final})
submission.to_csv('submission_result.csv', index=False)

Trial 1 Complete [00h 00m 00s]

Best val_loss So Far: None
Total elapsed time: 00h 00m 01s

Search: Running Trial #2

Value             |Best Value So Far |Hyperparameter
8                 |6                 |layers
128               |64                |units
relu              |relu              |activation
0                 |0.1               |dropout
adam              |adam              |optimizer
4.705e-05         |0.00011364        |learning_rate



Traceback (most recent call last):
  File "/home/exouser/.local/lib/python3.12/site-packages/keras_tuner/src/engine/base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/home/exouser/.local/lib/python3.12/site-packages/keras_tuner/src/engine/base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/exouser/.local/lib/python3.12/site-packages/keras_tuner/src/engine/tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/exouser/.local/lib/python3.12/site-packages/keras_tuner/src/engine/tuner.py", line 233, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
 

RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "/home/exouser/.local/lib/python3.12/site-packages/keras_tuner/src/engine/base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/home/exouser/.local/lib/python3.12/site-packages/keras_tuner/src/engine/base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/exouser/.local/lib/python3.12/site-packages/keras_tuner/src/engine/tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/exouser/.local/lib/python3.12/site-packages/keras_tuner/src/engine/tuner.py", line 233, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, **k

# Attempt 3 - Hint Given: `change X_train_proc to X_train`

In [2]:
# 1. Reproducibility
import random
random.seed(42)
import numpy as np
np.random.seed(42)
import tensorflow as tf
tf.random.set_seed(42)

# 2. Imports
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import json, time
import keras_tuner as kt
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model

# 3. Data Loading & Target Encoding
# Load datasets
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
submission_template = pd.read_csv('sample_submission.csv')

# Infer id_col and target_columns from sample submission
cols = submission_template.columns.tolist()
id_col = cols[0]
target_col = cols[1]

# Label encode target (binary classification)
le = LabelEncoder().fit(train_df[target_col].astype(str))
y_train = le.transform(train_df[target_col].astype(str)).astype(int)

# Prepare feature matrices
drop_cols = [id_col, target_col]
X = train_df.drop(columns=drop_cols, errors='ignore')
X_test = test_df.drop(columns=drop_cols, errors='ignore')
test_ids = test_df[id_col]

# 4. Feature Engineering: drop all-missing, high-cardinality categorical
# Drop columns with all missing
X = X.dropna(axis=1, how='all')
X_test = X_test[X.columns]
# Identify categorical and drop if cardinality >50
cat_cols = X.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()
high_card = [c for c in cat_cols if X[c].nunique() > 50]
X = X.drop(columns=high_card)
X_test = X_test.drop(columns=high_card)

# 5. Preprocessing Pipeline
d_numeric = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_features = X.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, d_numeric),
    ('cat', categorical_transformer, cat_features)
])

# Fit & transform
X_train_proc = preprocessor.fit_transform(X)
X_test_proc = preprocessor.transform(X_test)

# 6. Model Architecture for small dataset (n_features <100)
n_features = X_train_proc.shape[1]

# Keras-Tuner Model Definition
# Define early stopping and model checkpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

class MyHyperModel(kt.HyperModel):
    def build(self, hp):
        layers = hp.Int('layers', 2, 8)
        units = hp.Int('units', 64, 1024)
        act = hp.Choice('activation', ['relu'])
        drop = hp.Float('dropout', 0.0, 0.5)
        opt = hp.Choice('optimizer', ['adam'])
        lr = hp.Float('learning_rate', 1e-5, 0.01, sampling='log')

        inputs = Input(shape=(n_features,))
        x = inputs
        for _ in range(layers):
            x = Dense(units, activation=act)(x)
            x = Dropout(drop)(x)
        outputs = Dense(1, activation='sigmoid')(x)
        model = Model(inputs, outputs)
        model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
        return model

# Initialize the tuner
bs = 32  # batch size
ep = 20  # epochs

# Split the data for validation
X_train, X_val, y_train, y_val = train_test_split(X_train_proc, y_train, test_size=0.2, random_state=42)

# Search for the best hyperparameters
tuner = kt.BayesianOptimization(
    MyHyperModel(),
    objective='val_loss',
    max_trials=10,
    executions_per_trial=1,
    seed=42,
    overwrite=True,
    project_name='bayesian_tuner'
)

tuner.search(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=bs, epochs=ep,
    callbacks=[early_stopping, checkpoint]
)

# Build the best model
model = tuner.hypermodel.build(tuner.get_best_hyperparameters(1)[0])

# Retrain the model with original callbacks and data
start_time = time.time()
history = model.fit(
    X_train, y_train,  # Changed X_train_proc to X_train
    validation_data=(X_val, y_val),
    epochs=100, batch_size=bs,
    callbacks=[early_stopping, checkpoint],
    verbose=2
)
duration = time.time() - start_time

# 9. Evaluation & Logging
results = {
    'training_accuracy': history.history['accuracy'][-1],
    'training_loss': history.history['loss'][-1],
    'validation_accuracy': history.history['val_accuracy'][-1],
    'validation_loss': history.history['val_loss'][-1]
}
with open('results.json', 'w') as f:
    json.dump(results, f)

# 10. Prediction & Submission
raw_preds = model.predict(X_test_proc)
probs = raw_preds.flatten()
final = (probs > 0.5).astype(int)
submission = pd.DataFrame({id_col: test_ids.reset_index(drop=True), target_col: final})
submission.to_csv('submission_result.csv', index=False)

Trial 10 Complete [00h 02m 12s]
val_loss: 0.17434372007846832

Best val_loss So Far: 0.17095834016799927
Total elapsed time: 00h 22m 30s
Epoch 1/100


1467/1467 - 12s - 8ms/step - accuracy: 0.9132 - loss: 0.2477 - val_accuracy: 0.9390 - val_loss: 0.1897
Epoch 2/100


1467/1467 - 6s - 4ms/step - accuracy: 0.9337 - loss: 0.2107 - val_accuracy: 0.9447 - val_loss: 0.1804
Epoch 3/100


1467/1467 - 6s - 4ms/step - accuracy: 0.9372 - loss: 0.2016 - val_accuracy: 0.9458 - val_loss: 0.1770
Epoch 4/100


1467/1467 - 6s - 4ms/step - accuracy: 0.9397 - loss: 0.1966 - val_accuracy: 0.9462 - val_loss: 0.1758
Epoch 5/100


...

1467/1467 - 6s - 4ms/step - accuracy: 0.9430 - loss: 0.1909 - val_accuracy: 0.9486 - val_loss: 0.1723
Epoch 8/100
1467/1467 - 6s - 4ms/step - accuracy: 0.9426 - loss: 0.1899 - val_accuracy: 0.9465 - val_loss: 0.1750
Epoch 9/100


1467/1467 - 5s - 4ms/step - accuracy: 0.9436 - loss: 0.1879 - val_accuracy: 0.9488 - val_loss: 0.1716
Epoch 10/100
1467/1467 - 5s - 4ms/step - accuracy: 0.9439 - loss: 0.1870 - val_accuracy: 0.9477 - val_loss: 0.1724
Epoch 11/100
1467/1467 - 5s - 4ms/step - accuracy: 0.9443 - loss: 0.1868 - val_accuracy: 0.9490 - val_loss: 0.1716
Epoch 12/100
1467/1467 - 5s - 4ms/step - accuracy: 0.9449 - loss: 0.1847 - val_accuracy: 0.9475 - val_loss: 0.1741
Epoch 13/100


1467/1467 - 6s - 4ms/step - accuracy: 0.9443 - loss: 0.1864 - val_accuracy: 0.9497 - val_loss: 0.1710
Epoch 14/100
1467/1467 - 5s - 4ms/step - accuracy: 0.9448 - loss: 0.1856 - val_accuracy: 0.9483 - val_loss: 0.1722
Epoch 15/100
1467/1467 - 5s - 3ms/step - accuracy: 0.9454 - loss: 0.1838 - val_accuracy: 0.9491 - val_loss: 0.1711
Epoch 16/100
1467/1467 - 5s - 4ms/step - accuracy: 0.9457 - loss: 0.1827 - val_accuracy: 0.9497 - val_loss: 0.1720
Epoch 17/100
1467/1467 - 7s - 5ms/step - accuracy: 0.9460 - loss: 0.1823 - val_accuracy: 0.9485 - val_loss: 0.1726
Epoch 18/100
1467/1467 - 6s - 4ms/step - accuracy: 0.9454 - loss: 0.1831 - val_accuracy: 0.9488 - val_loss: 0.1723
Epoch 19/100
1467/1467 - 6s - 4ms/step - accuracy: 0.9459 - loss: 0.1822 - val_accuracy: 0.9492 - val_loss: 0.1731
Epoch 20/100
1467/1467 - 6s - 4ms/step - accuracy: 0.9458 - loss: 0.1824 - val_accuracy: 0.9499 - val_loss: 0.1713
Epoch 21/100
1467/1467 - 5s - 4ms/step - accuracy: 0.9464 - loss: 0.1817 - val_accuracy: 0.94

2025-07-17 16:45:18.089385: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_18', 4 bytes spill stores, 4 bytes spill loads

2025-07-17 16:45:18.129666: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_18', 60 bytes spill stores, 60 bytes spill loads

2025-07-17 16:45:18.299965: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_18', 176 bytes spill stores, 176 bytes spill loads

2025-07-17 16:45:18.309911: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_18', 136 bytes spill stores, 136 bytes spill loads

2025-07-17 16:45:18.409240: I external/local_x

1222/1222 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step


In [ ]:
print(duration)

138.574481
